##### Import packages and LLM configurations

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
from tqdm import trange
from together import Together

# Set up the API key for Together AI
api_key="api_key_here" 
client = Together(api_key=api_key)  
# This model's maximum context length is 131072(128K) tokens. requested 134193 tokens 34193 in the messages, 100000 in the completion
def generate_response(system, prompt):
    completion = client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
        messages=[{"role":"system","content":system},
                  {"role":"user","content":prompt}],
        max_tokens=131072-34193, #1024
        temperature=0.75,
        top_p=0.95,
        repetition_penalty=1,
        stop=["<|eot_id|>","<|eom_id|>"],
        stream=False,
        seed=42
    )
    return completion.choices[0].message.content.strip()

##### Read and randomly split dataset

In [ ]:
df = pd.read_excel('novel_generate_v4.xlsx', keep_default_na=False)
df = df.iloc[:1000].sample(frac=1, random_state=11, ignore_index=True)
df_train = df.iloc[:400]
df_val = df.iloc[400:600]
df_test = df.iloc[600:]

##### Retrieve 5/10/20/38 samples in training dataset

In [ ]:
df_train_selected = pd.concat([
    df_train[df_train.label==1],
    df_train[df_train.label==0].sample(38, random_state=11)
]).sample(frac=1, random_state=11, ignore_index=True)
print(df_train_selected.value_counts('label'))

label
0    38
1    38
dtype: int64


##### Generate reasons for each training samples (common needs as example below)

In [ ]:
system = f'You are a helpful customer review expert'
for i in trange(106,1000):
    prompt = f'''Given customer review: {df.text[i]}
    Summarize the reason within 10 words why the input is a review with common needs rather than review with novel needs.
    '''
    reasons = generate_response(system, prompt)
    print(reasons)
    print('='*60)
    df.novel_reason[i] = reasons

##### Generate and see the prompt including 10 samples

In [ ]:
num_sample = 10
df_train = df_train_selected.sample(num_sample, random_state=11, ignore_index=True)
print(df_train.value_counts('label'))
pair = ''
for i in range(1,num_sample):
    pair = pair + f'''Customer review: {df_train.text[i]} 
Expected output: {df_train.label[i]}, because the laptop has {df_train.novel_reason[i]}
'''
prompt = f'''Given customer review: {df_train.text[0]}
And its expected output, 0 for review with common needs and 1 for review with novel needs: {df_train.label[0]}, because the laptop has {df_train.novel_reason[0]}
{pair}Explain the reasons why the input corresponds to the given expected output one by one. The reasons should be placed within tags <reason> and </reason>.
'''
print(prompt)

label
1    6
0    4
dtype: int64
Given customer review: The keyboard layout of this computer is pretty intuitive, but the lack of backlighting makes it hard to use in dimly lit conditions. A feature I would love to see is smart keyboard lighting that adjusts based on the room's light conditions.
And its expected output, 0 for review with common needs and 1 for review with novel needs: 1, because the laptop has smart keyboard lighting that adjusts based on the room's light conditions
Customer review: This laptop has pretty much everything I was looking for. It boots up very quickly, processes quickly, and it lightweight. The battery life is pretty great. My only issue is with the touchpad. It can be sensitive and aggravating sometimes, probably because the left and right-click buttons are part of the touchpad. It would be better if the screen is foldable, flexible or even expandable so I can enjoy both the big sceen and small weight at the mean time. 
Expected output: 1, because the lap

##### Generate reasons of 10 samples

In [ ]:
system = f'You are a helpful customer review expert'
reasons = generate_response(system, prompt)
print(reasons)

I'll go through each review and explain the reasons why the input corresponds to the given expected output.

**1. Review:** To begin with, I received the product yesterday and I have already made the decision to return it. From the moment I turned it on after setting it up, I encountered a black screen twice. I had to continuously reboot the laptop. Additionally, the computer's performance was much slower than my expectations, with noticeable lags. Today was my first day using it, and despite needing a new laptop urgently, I do not wish to keep this one. I was specifically looking for a laptop with built-in document reader, scanner, and photo editor capabilities.
**Expected Output:** 1
<reason>The review mentions specific features the customer was looking for in a laptop, namely built-in document reader, scanner, and photo editor capabilities, which indicates a novel or unique requirement.</reason>

**2. Review:** I am so glad I found this product that I can use. User friendly very con

##### Summarize all reasons

In [ ]:
system = f'You are a helpful customer review expert'
prompt = f'''{reasons}
Given input and expected output pairs, along with the reason for generated outputs, provide a summarized common reason applicable to all cases within tags <summary> and </summary>.
The summary should explain the underlying principles, logic or methodology governing the relationship between the inputs and corresponding outputs. Avoid mentioning any specific details, numbers, or entities from the individual examples, and aim for a generalized explanation.
'''
res = generate_response(system, prompt)
print(res)

<summary>
The common reason governing the relationship between the inputs and corresponding outputs is the presence or absence of novel or unique requirements mentioned in the reviews. Outputs are marked as 1 when the review contains a mention of an innovative feature, idea, or functionality that is not commonly found in laptops, such as advanced material usage or smart lighting systems. On the other hand, outputs are marked as 0 when the review only discusses general pros and cons, common issues, or standard features, without indicating any unique or novel requirements. This methodology suggests that the output classification is based on the reviewer's expression of creative or unconventional expectations from the product, as opposed to standard or typical expectations.
</summary>


##### Baselines for 0/5/10 samples without reason summaries

In [ ]:
no_sum_tem075_seed42_tem075 = ""

zeroshot_tem075_seed42_tem075 = "Return an output of 0 for reviews with common needs or an output of 1 for reviews with novel needs, nothing else"

fiveshot_seed11_tem075_seed42_tem075 = """
Customer review: The keyboard layout of this computer is pretty intuitive, but the lack of backlighting makes it hard to use in dimly lit conditions. A feature I would love to see is smart keyboard lighting that adjusts based on the room's light conditions.
Expected output: 1
Customer review: This laptop has pretty much everything I was looking for. It boots up very quickly, processes quickly, and it lightweight. The battery life is pretty great. My only issue is with the touchpad. It can be sensitive and aggravating sometimes, probably because the left and right-click buttons are part of the touchpad. It would be better if the screen is foldable, flexible or even expandable so I can enjoy both the big sceen and small weight at the mean time. 
Expected output: 1
Customer review: Very fast! The laptop works great, and everything loads super fast. 10/10 would recommend. An eye-tracking system that allows the customer to scroll and select items on the screen without having to use their hands 
Expected output: 1
Customer review: I liked 
Expected output: 0
Customer review: Streaming isn't great but it performs Good overall 
Expected output: 0
Return an output of 0 for reviews with common needs or an output of 1 for reviews with novel needs, nothing else
"""

tenshot_seed11_tem075_seed42_tem075 = """
Customer review: The keyboard layout of this computer is pretty intuitive, but the lack of backlighting makes it hard to use in dimly lit conditions. A feature I would love to see is smart keyboard lighting that adjusts based on the room's light conditions.
Expected output: 1
Customer review: This laptop has pretty much everything I was looking for. It boots up very quickly, processes quickly, and it lightweight. The battery life is pretty great. My only issue is with the touchpad. It can be sensitive and aggravating sometimes, probably because the left and right-click buttons are part of the touchpad. It would be better if the screen is foldable, flexible or even expandable so I can enjoy both the big sceen and small weight at the mean time. 
Expected output: 1
Customer review: Very fast! The laptop works great, and everything loads super fast. 10/10 would recommend. An eye-tracking system that allows the customer to scroll and select items on the screen without having to use their hands 
Expected output: 1, because the laptop has eye-tracking system that allows the customer to scroll and select items on the screen
Customer review: I liked 
Expected output: 0
Customer review: Streaming isn't great but it performs Good overall 
Expected output: 0
Customer review: The backlit keyboard offers a superior typing experience with its bright and well-lit keys. It features a dual-surface construction that provides a combination of rigid and soft textures, allowing for varied and comfortable typing experiences. 
Expected output: 1
Customer review: The display on this computer is stunning, but the touchpad feels a bit outdated. I'd love it if they could integrate gesture recognition technology for a seamless and intuitive navigation experience. 
Expected output: 1
Customer review: The build quality of this computer is truly commendable, with a robust frame that feels like it can withstand a minor earthquake. However, the bland design leaves me yearning for more. If only they could introduce a customizable exterior that allows users to change the color and texture to suit their personal style! 
Expected output: 1
Customer review: Has worked well and would recommend it to a friend. 
Expected output: 0
Customer review: The hinges don't leave me confident in the durability either. 
Expected output: 0
Return an output of 0 for reviews with common needs or an output of 1 for reviews with novel needs, nothing else
"""

system = tenshot_seed11_tem075_seed42_tem075
output = []
for i in trange(600,1000):
    prompt = f'''
    {df_test.text[i]}
    '''
    res = generate_response(system, prompt)
    print("Correct label:", df_test.label[i])
    print(res)
    print('='*60)
    output.append(int(res) if len(res)==1 else 0)
pd.DataFrame(output).to_excel('df_test_10shot_seed11_tem075_seed42_tem075.xlsx', index=False)
print(classification_report(df_test.label[:], output, target_names = ['Common','Unique'], digits=4))
print(confusion_matrix(df_test.label[:], output))

100%|██████████| 142/142 [02:45<00:00,  1.17s/it]


##### Summaries of different settings

In [ ]:
reason_sum_without_common_reason_tem075_seed42_tem075_train5_seed11 = "The common reason governing the relationship between the inputs and corresponding outputs is the presence or absence of novel or innovative feature suggestions in the reviews. Outputs of 1 are generated when reviewers mention unique, futuristic, or uncommon features they would like to see, indicating a desire for innovative solutions to problems they encountered. In contrast, outputs of 0 are generated when reviewers do not suggest any novel features or improvements, instead focusing on common expectations or issues without providing creative solutions. The underlying principle is to identify reviewers who demonstrate a need for cutting-edge technology or innovative design, as opposed to those who simply report on standard features or common issues."

reason_sum_without_common_reason_tem075_seed42_tem075_train10_seed11 = "The common reason governing the relationship between the inputs and corresponding outputs is the presence or absence of novel feature suggestions. The expected output is 1 when the review mentions a unique or innovative feature that is not typically found in the product, indicating a novel need or suggestion. On the other hand, the expected output is 0 when the review only mentions common issues, concerns, or needs that are already well-established, without proposing any new or unique solutions. The underlying principle is to identify whether the review contains a creative or unusual suggestion that goes beyond the typical features or requirements of the product."

reason_sum_without_common_reason_tem075_seed42_tem075_train20_seed11 = "The common reason governing the relationship between the inputs and corresponding outputs is the presence or absence of novel feature suggestions or needs in the customer reviews. The output is 1 when the review expresses a desire for an innovative solution, a feature that is not commonly found in laptops, or a suggestion that goes beyond the standard features of a laptop. On the other hand, the output is 0 when the review only mentions common issues, praises existing features, or evaluates the laptop based on its standard features without suggesting any novel solutions. This indicates that the reviewer has common needs and is not looking for anything beyond what is typically expected from a laptop. The underlying principle is to identify whether the reviewer is simply evaluating the laptop based on its existing features or if they are looking for something new and innovative. This distinction is crucial in determining whether the reviewer has common or novel needs."

reason_sum_without_common_reason_tem075_seed42_tem075_train5_seed22 = "The underlying principle governing the relationship between the inputs and corresponding outputs is the distinction between common and novel needs. Reviews that express satisfaction or dissatisfaction with typical aspects of a product, such as its performance, usability, price, and reliability, are classified as corresponding to common needs. On the other hand, reviews that express desires for innovative, futuristic, or unconventional features, or that highlight specific requirements that go beyond typical expectations, are classified as corresponding to novel needs. This distinction is based on the idea that customers have certain baseline expectations for a product, and that reviews that fall within these expectations are considered common, while reviews that exceed or deviate from these expectations are considered novel."

reason_sum_without_common_reason_tem075_seed42_tem075_train10_seed22 = "The common reason governing the relationship between the inputs and corresponding outputs is the presence or absence of novel or unique requirements mentioned in the reviews. Outputs are marked as 1 when the review contains a mention of an innovative feature, idea, or functionality that is not commonly found in laptops, such as advanced material usage or smart lighting systems. On the other hand, outputs are marked as 0 when the review only discusses general pros and cons, common issues, or standard features, without indicating any unique or novel requirements. This methodology suggests that the output classification is based on the reviewer's expression of creative or unconventional expectations from the product, as opposed to standard or typical expectations."

reason_sum_without_common_reason_tem075_seed42_tem075_train20_seed22 = "The common reason governing the relationship between the inputs and corresponding outputs is the presence or absence of novel or unique features mentioned in the customer reviews. The output is 1 when the review expresses a desire for or mentions a feature that is not commonly found in laptops, such as advanced materials, AI-powered settings, or innovative design elements. On the other hand, the output is 0 when the review discusses general satisfaction, common issues, or typical features found in laptops, such as processor speed, memory, and display quality. The underlying principle is that the model is looking for reviews that indicate the customer is thinking outside the box and envisioning new or improved features that go beyond the standard specifications of a laptop."

reason_sum_without_common_reason_tem075_seed42_tem075_train5_seed44 = "The common reason governing the relationship between the inputs and corresponding outputs is the presence or absence of novel or innovative ideas in the reviews. The outputs are determined by whether the reviewer is simply stating common issues, expectations, or features (resulting in a 0 output) or if they are suggesting new, creative, or outside-the-box ideas for improvement (resulting in a 1 output). The underlying principle is to identify the level of inventiveness or originality in the reviewer's comments, with a higher output indicating a more imaginative or pioneering suggestion."

reason_sum_without_common_reason_tem075_seed42_tem075_train10_seed44 = """The common reason governing the relationship between the inputs and corresponding outputs is the presence or absence of a novel feature suggestion. The output is 1 when the customer review expresses a desire for a new, innovative, or non-standard feature that is not typically found in most laptops. This feature suggestion is often introduced by phrases such as "how about," "I imagine," or "a computer with," which indicate that the customer is proposing an idea that goes beyond the current capabilities of the laptop. On the other hand, the output is 0 when the customer review only states their satisfaction or dissatisfaction with the laptop's existing features, without suggesting any new or innovative ideas. The underlying principle is to identify whether the customer is simply evaluating the product as it is or proposing a new direction for future development."""

reason_sum_without_common_reason_tem075_seed42_tem075_train20_seed44 = "The common reason governing the relationship between the inputs and corresponding outputs is the presence or absence of novel feature suggestions in the customer reviews. The outputs are determined by whether the customer review expresses a desire for a new or innovative feature that is not a standard feature in most laptops. If a customer review mentions a specific feature that is not commonly found in laptops, the output is 1, indicating that the review contains a novel feature suggestion. On the other hand, if the review only reports problems with existing features, expresses satisfaction with existing features, or suggests alternative products without mentioning new features, the output is 0, indicating that the review does not contain a novel feature suggestion. The underlying logic is to identify whether the customer is simply providing feedback on the product's current state or if they are proposing new ideas for improvement. This distinction is crucial in determining the output, as it helps to categorize the reviews based on their content and the customer's intentions."

In [ ]:
reason_sum_tem075_seed42_tem0 = "The common reason governing the relationship between the inputs and corresponding outputs is the presence or absence of novel needs or features expressed by the customer in their review. The output is 1 when the customer mentions a specific, unique, or innovative feature that is not commonly found in laptops, indicating a novel need. On the other hand, the output is 0 when the customer expresses common sentiments, concerns, or preferences that are typical in laptop reviews, without mentioning any specific novel needs or features. The underlying principle is to identify whether the customer's review contains a distinctive or original idea that goes beyond the usual expectations or features of a laptop."

reason_sum_tem075_seed42_tem025 = "The common reason governing the relationship between the inputs and corresponding outputs is the presence or absence of novel needs or features expressed by the customer in their review. The output is 1 when the customer mentions a specific, uncommon, or innovative feature that is not typically found in laptops, indicating a novel need. On the other hand, the output is 0 when the customer expresses common sentiments, concerns, or preferences that are typical of laptop users, without mentioning any specific novel needs or features. The underlying principle is to identify whether the customer's review contains a unique or creative suggestion for a feature that goes beyond the standard expectations of a laptop."

reason_sum_tem075_seed42_tem05 = """The common reason governing the relationship between the inputs and corresponding outputs is the presence or absence of novel feature requests or needs expressed by the customers in their reviews. The output is 1 when a customer mentions a specific, uncommon, or innovative feature that is not typically found in laptops, indicating a novel need. On the other hand, the output is 0 when a customer expresses common sentiments, satisfaction, or dissatisfaction with existing features, or mentions common issues or concerns without suggesting new or unique features. This distinction is based on the idea that novel needs or feature requests are those that go beyond the standard or typical features and functionalities of laptops, and are often characterized by words or phrases such as "I wish," "I would love to see," or "it would be great if." The underlying principle is to identify and distinguish between routine or expected feedback and feedback that suggests new or innovative ideas for laptop design or functionality."""

reason_sum_tem075_seed42_tem1 = """The common reason governing the relationship between the inputs and corresponding outputs is the presence or absence of novel feature suggestions or needs expressed by the customers in their reviews. The output is 1 when a customer mentions a specific novel feature or need that is not commonly found in laptops, indicating a unique or innovative idea. On the other hand, the output is 0 when a customer expresses common sentiments, concerns, or satisfaction with existing features, but does not mention any novel needs or features. The underlying principle is that the model is trained to identify and differentiate between common and novel ideas, with a focus on recognizing creative or innovative suggestions that can potentially enhance or improve the product. The logic governing the outputs is based on the model's ability to analyze the customer's feedback, identify patterns and keywords, and determine whether the customer is expressing a unique or common idea. The methodology employed in generating the outputs involves a combination of natural language processing (NLP) and machine learning algorithms, which enable the model to understand the context and content of the customer's review, and make informed decisions about the presence or absence of novel feature suggestions."""

reason_sum_tem0_seed42_tem075 = "The common reason governing the relationship between the inputs and corresponding outputs is the presence or absence of novel feature suggestions in the customer reviews. The outputs are generated based on whether the customer review contains a suggestion for a new or innovative feature that is not commonly found in laptops. If a review suggests a unique feature, the output is 1, indicating the presence of a novel idea. On the other hand, if the review only mentions common issues, general opinions, or does not provide any specific feature suggestions, the output is 0, indicating the absence of a novel idea. This underlying principle is based on the idea that novel feature suggestions can provide valuable insights for product development and improvement, and can be used to identify potential areas for innovation."

reason_sum_tem025_seed42_tem075 = "The common reason governing the relationship between the inputs and corresponding outputs is the presence or absence of novel needs or features expressed by the customer. Outputs labeled as 1 indicate that the customer has mentioned a unique or innovative feature that is not commonly found in laptops, such as advanced keyboard functionalities, immersive gaming capabilities, or personalized customization options. On the other hand, outputs labeled as 0 signify that the customer's review is focused on common sentiments, complaints, or preferences that do not introduce new or unusual feature requests. The underlying logic is to identify whether the customer's feedback contains creative or unconventional ideas that could potentially inspire new product developments or improvements."

reason_sum_tem05_seed42_tem075 = "The common reason governing the relationship between the inputs and corresponding outputs is the presence or absence of novel feature requests or needs expressed by the customers in their reviews. The output is 1 when a customer mentions a specific, uncommon, or innovative feature that is not typically found in laptops, such as advanced display technologies, unique keyboard designs, or futuristic capabilities. On the other hand, the output is 0 when a customer expresses common sentiments, concerns, or use cases that are typical for laptops, without mentioning any specific novel needs or features. The underlying principle is to identify and distinguish between ordinary, everyday customer feedback and extraordinary, forward-thinking suggestions that could potentially drive innovation in laptop design and functionality."

reason_sum_tem1_seed42_tem075 = "The common reason governing the relationship between the inputs and corresponding outputs is the presence or absence of novel needs or feature requests in the customer reviews. The output is 1 when the review expresses a unique, innovative, or uncommon requirement that is not typically found in laptops, such as advanced technologies, customizable features, or specialized capabilities. On the other hand, the output is 0 when the review only mentions general expectations, common features, technical issues, or warranty concerns, without expressing any novel needs. This suggests that the underlying logic is based on identifying reviews that showcase creative, forward-thinking, or specialized demands that go beyond standard laptop functionalities."

In [ ]:
reason_sum_with_common_reason_tem0_seed42_tem0='The underlying principle governing the relationship between the inputs and corresponding outputs is the presence or absence of novel needs or features mentioned by the reviewer. If the reviewer mentions a specific feature or need that goes beyond the common features of a laptop, the output is 1. If the reviewer mentions common issues, basic product features, or lacks specific details, the output is 0. This suggests that the model is designed to identify and distinguish between common and novel needs or features mentioned in customer reviews.'

reason_sum_without_common_reason_tem0_seed421_tem0="The common reason for the generated outputs is that the model is identifying whether a customer review suggests a novel or innovative feature that is not commonly found in laptops. The model is trained to recognize language patterns that indicate a customer is proposing a new idea or solution that goes beyond the standard features and functionalities of a laptop. The output is 1 when the review suggests a novel feature, and 0 when the review only mentions common issues, general opinions, or standard features. The underlying principle is to detect the customer's intent to innovate or improve the product, rather than simply providing feedback on its current state."

reason_sum_without_common_reason_tem0_seed42_tem0 ="The common reason for the generated outputs is that the model is identifying whether a customer review suggests a novel or innovative feature that is not commonly found in laptops. The model is trained to recognize language patterns that indicate a customer is proposing a new idea or solution that goes beyond the standard features and functionalities of a laptop. The output is 1 when the review suggests a novel feature, and 0 when the review only mentions common issues, general opinions, or standard features. The underlying principle is to detect the customer's intent to innovate or improve the product, rather than simply providing feedback on its current state."

reason_sum_without_common_reason_tem1_seed42_tem1="The common reason governing the relationship between the inputs and corresponding outputs is the presence or absence of novel needs or feature requests in the customer reviews. The output is 1 when the review expresses a unique, innovative, or uncommon need or feature that is not typically found in laptops, indicating a novel requirement. In contrast, the output is 0 when the review only mentions common expectations, issues, or features that are already present in laptops, without expressing any new or unusual needs. The underlying principle is to identify whether the customer is simply discussing existing features, common problems, or general statements, or if they are proposing a new idea, improvement, or innovation that goes beyond the current state of laptops. This distinction allows for the detection of novel needs, which can provide valuable insights for product development, improvement, and innovation."

reason_sum_without_common_reason_tem075_seed42_tem075 = "The common reason governing the relationship between the inputs and corresponding outputs is the presence or absence of novel feature requests or needs expressed by the customers in their reviews. The output is 1 when a customer mentions a specific, uncommon, or innovative feature that is not typically found in laptops, such as advanced display technologies, intelligent assistants, or modular design elements. On the other hand, the output is 0 when a customer expresses common sentiments, concerns, or experiences related to laptop performance, design, or functionality that do not involve novel or unusual feature requests. The underlying principle is to identify and distinguish between ordinary customer feedback and extraordinary suggestions that could potentially lead to new product developments or innovations."

reason_sum_without_common_reason_tem05_seed42_tem05="""The common reason governing the relationship between the inputs and corresponding outputs is the presence or absence of novel or specific needs expressed by the customer. Outputs labeled as "1" indicate that the customer has expressed a unique or innovative feature request that is not commonly found in laptops, such as advanced display technologies, intelligent assistants, or modular designs. On the other hand, outputs labeled as "0" indicate that the customer has expressed common sentiments, concerns, or preferences that are typical of laptop users, such as satisfaction with performance, issues with durability, or expectations for battery life. The underlying principle is to identify whether the customer's feedback contains a novel or specific need that goes beyond the usual expectations and common features of laptops."""

reason_sum_without_common_reason_tem025_seed42_tem025 = "The common reason governing the relationship between the inputs and corresponding outputs is the presence or absence of novel needs or features expressed by the customer in their review. The output is 1 when the customer mentions a unique or innovative feature that is not commonly found in laptops, indicating a novel need. On the other hand, the output is 0 when the customer expresses common sentiments, complaints, or preferences that do not mention any new or unusual features. This suggests that the underlying logic is based on identifying and distinguishing between ordinary and extraordinary customer needs, with the former being typical and expected, and the latter being unusual and innovative."

##### Hard Voting in different temperature settings

In [ ]:
excel_list = ['df_test_38_without_common_reason_seed42_tem0.xlsx',
            'df_test_38_without_common_reason_seed42_tem025.xlsx',
            'df_test_38_without_common_reason_seed42_tem05.xlsx',
            'df_test_38_without_common_reason_seed42_tem075.xlsx',
            'df_test_38_without_common_reason_seed42_tem1.xlsx']
df_vote = pd.DataFrame()
for excel in excel_list:
    df_vote = pd.concat([df_vote,pd.read_excel(excel)],axis=1)
df_vote=pd.concat([df_vote,pd.DataFrame(df_test.label.values)],axis=1)
df_vote.columns=["0","0.25","0.5","0.75","1","label"]
df_vote["hard_vote"] = [-1]*len(df_vote)
for i in range(len(df_vote)):
	df_vote.iloc[i,-1] = 1 if np.sum(df_vote.iloc[i,:len(excel_list)])>4 else 0
print('hard vote')
print(classification_report(df_vote.label, df_vote.hard_vote, target_names = ['Common','Unique'], digits=4))
print(confusion_matrix(df_vote.label, df_vote.hard_vote))
df_vote

hard vote
              precision    recall  f1-score   support

      Common     0.9693    0.9914    0.9802       350
      Unique     0.9286    0.7800    0.8478        50

    accuracy                         0.9650       400
   macro avg     0.9489    0.8857    0.9140       400
weighted avg     0.9642    0.9650    0.9637       400

[[347   3]
 [ 11  39]]


,0,0.25,0.5,0.75,1,label,hard_vote
0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...
395,0,0,0,0,0,0,0
396,1,1,1,1,1,1,1
397,1,1,1,1,1,1,1
398,0,0,0,0,0,0,0
